In [ ]:
"""This demo program solves Poisson's equation

    - div grad u(x, y) = f(x, y)

on the unit square with source f given by

    f(x, y) = -100*exp(-((x - 0.5)^2 + (y - 0.5)^2)/0.02)

and boundary conditions given by

    u(x, y)     = u0 on x = 0 and x = 1

where

    u0 = x + 0.25*sin(2*pi*x)

using a discontinuous Galerkin formulation (interior penalty method).
"""

from dolfin import *


# FIXME: Make mesh ghosted
parameters["ghost_mode"] = "shared_facet"

# Define class marking Dirichlet boundary (x = 0 or x = 1)
class DirichletBoundary(SubDomain):
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0]*(1 - x[0]), 0)

# Create mesh and define function space
mesh = UnitSquareMesh (24, 24)
V = FunctionSpace(mesh, 'DG', 1)

# Define test and trial functions
u = TrialFunction(V)
v = TestFunction(V)

# Define normal vector and mesh size
n = FacetNormal(mesh)
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2

# Define the source term f, Dirichlet term u0 and Neumann term g
f = Expression('-100.0*exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)) / 0.02)', degree=2)
u0 = Expression('x[0] + 0.25*sin(2*pi*x[1])', degree=2)

# Mark facets of the mesh
boundaries = MeshFunction('size_t', mesh, mesh.topology().dim()-1, 0)
DirichletBoundary().mark(boundaries, 1)

# Define outer surface measure aware of Dirichlet and Neumann boundaries
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

# Define parameters
alpha = 4.0
gamma = 80.0

# Define variational problem
a = dot(grad(v), grad(u))*dx \
   - dot(avg(grad(v)), jump(u, n))*dS \
   - dot(jump(v, n), avg(grad(u)))*dS \
   + alpha/h_avg*dot(jump(v, n), jump(u, n))*dS \
   - dot(grad(v), u*n)*ds(1) \
   - dot(v*n, grad(u))*ds(1) \
   + (gamma/h)*v*u*ds(1)
L = v*f*dx - u0*dot(grad(v), n)*ds(1) + (gamma/h)*u0*v*ds(1)

# Compute solution
u = Function(V)
solve(a == L, u)
print("Solution vector norm (0): {!r}".format(u.vector().norm("l2")))

In [ ]:
""" Steady state advection-diffusion equation,
discontinuous formulation using full upwinding.

Implemented in python from cpp demo by Johan Hake.
"""

from dolfin import *
import matplotlib.pyplot as plt


# FIXME: Make mesh ghosted
parameters["ghost_mode"] = "shared_facet"

class DirichletBoundary (SubDomain):
    def inside (self, x, on_boundary):
        return abs(x[0] - 1.0) < DOLFIN_EPS and on_boundary

# Load mesh
mesh = UnitSquareMesh (40, 40)

# Defining the function spaces
V_dg = FunctionSpace (mesh, "DG", 1)
V_cg = FunctionSpace (mesh, "CG", 1)
V_u  = VectorFunctionSpace (mesh, "CG", 2)

# Create velocity Function
# u = Constant((-1,-0.4))
u = interpolate ( Constant (("-1.", "-0.4")), V_u)

# Test and trial functions
v   = TestFunction (V_dg)
phi = TrialFunction (V_dg)

# Diffusivity
kappa = Constant (0.0)

# Source term
f = Constant (0.0)

# Penalty term
alpha = Constant (5.0)

# Mesh-related functions
n = FacetNormal (mesh)
h = CellDiameter (mesh)
h_avg = (h ('+') + h ('-'))/2

# ( dot(v, n) + |dot(v, n)| )/2.0
un = (dot (u, n) + abs (dot (u, n)))/2.0

# Bilinear form (advection-diffusion equation)
# a_int = dot(grad(v), kappa*grad(phi) - u*phi)*dx
# a_fac = kappa*(alpha/h('+'))*dot(jump(v, n), jump(phi, n))*dS \
#       - kappa*dot(avg(grad(v)), jump(phi, n))*dS \
#       - kappa*dot(jump(v, n), avg(grad(phi)))*dS
# a_vel = dot(jump(v), un('+')*phi('+') - un('-')*phi('-') )*dS  + dot(v, un*phi)*ds
# a = a_int + a_fac + a_vel

# Bilinear form (advection equation)
a_int = dot (grad (v), - u*phi)*dx
a_vel = dot (jump(v), un ('+')*phi ('+') - un ('-')*phi ('-') )*dS  + dot(v, un*phi)*ds
a = a_int + a_vel

# Linear form
L = v*f*dx

# Set up boundary condition (apply strong BCs)
g = Expression ("sin (pi*5.0*x[1])", degree = 2)
bc = DirichletBC (V_dg, g, DirichletBoundary (), "geometric")

# Solution function
phi_h = Function (V_dg)

# Assemble and apply boundary conditions
A = assemble (a)
b = assemble (L)
bc.apply (A, b)

# Solve system
solve (A, phi_h.vector (), b)

# Project solution to a continuous function space
up = project (phi_h, V = V_cg)

# Discontinuous function space
# up = phi_h

file = File ("temperature.pvd")
file << up

# Plot solution
plot (up)
plt.show ()

In [ ]:
from dolfin import *
import matplotlib.pyplot as plt

# FIXME: Make mesh ghosted
parameters["ghost_mode"] = "shared_facet"

# Dirichlet boundary
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < (DOLFIN_EPS + 1.0)and on_boundary
        
# Mesh
mesh = UnitIntervalMesh(50)

# Function spaces
V_dg = FunctionSpace(mesh, "DG", 1)
V_cg = FunctionSpace(mesh, "CG", 1)
V_u  = FunctionSpace(mesh, "CG", 2) 

# Velocity 
u_const = Constant(0.5)
u = interpolate(u_const, V_u)

# Test and trial functions
v   = TestFunction(V_dg)
phi = TrialFunction(V_dg)

# Source term
f = Constant(0.0)

# Penalty term
alpha = Constant(5.0)

# Mesh-related functions
n = FacetNormal(mesh)
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2

# Upwind 
uv = as_vector ((u, ))
un = (dot (uv, n) - abs (dot (uv, n)) )/2

# Bilinear form (advection equation)
a_int = inner (Dx (v, 0), - u*phi)*dx
a_vel = dot (jump(v), un ('+')*phi ('+') - un ('-')*phi ('-') )*dS  + dot (v, un*phi)*ds
a = a_int + a_vel

# Linear form
L = v*f*dx

# Set up boundary condition (apply strong BCs)
g = Expression ("sin(pi*5.0*x[1])", degree=2)
bc = DirichletBC (V_dg, g, DirichletBoundary(), "geometric")

# Solution function
phi_h = Function(V_dg)


solve(a == L, phi_h)

print("Solution vector norm (0): {!r}".format(phi_h.vector().norm("l2")))

# # Assemble and apply boundary conditions
# A = assemble(a)
# b = assemble(L)
# bc.apply(A, b)

# # Mark facets of the mesh

# boundaries = MeshFunction('size_t', mesh, 0)
# DirichletBoundary().mark(boundaries, 1)

# bc = DirichletBC (V_dg, g, DirichletBoundary (), "geometric")

# # Solution function
# phi_h = Function (V_dg)

# # Assemble and apply boundary conditions
# A = assemble (a)
# b = assemble (L)
# bc.apply (A, b)

In [ ]:

from dolfin import *
import matplotlib.pyplot as plt

# FIXME: Make mesh ghosted
parameters["ghost_mode"] = "shared_facet"

class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0] - 1.0) < DOLFIN_EPS and on_boundary

# Load mesh
mesh = UnitSquareMesh(50,50)

# Defining the function spaces
V_dg = FunctionSpace(mesh, "DG", 1)
V_cg = FunctionSpace(mesh, "CG", 1)
V_u  = VectorFunctionSpace(mesh, "CG", 2)

# Create velocity Function
u = Constant((3,3))

# Test and trial functions
v   = TestFunction(V_dg)
phi = TrialFunction(V_dg)

# Diffusivity
kappa = Constant(0.0)

# Source term
f = Constant(0.0)

# Penalty term
alpha = Constant(5.0)

# Mesh-related functions
n = FacetNormal(mesh)
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2

# ( dot(v, n) + |dot(v, n)| )/2.0
un = (dot(u, n) + abs(dot(u, n)))/2.0

# Bilinear form
a_int = dot(grad(v), kappa*grad(phi) - u*phi)*dx

a_fac = kappa*(alpha/h('+'))*dot(jump(v, n), jump(phi, n))*dS \
      - kappa*dot(avg(grad(v)), jump(phi, n))*dS \
      - kappa*dot(jump(v, n), avg(grad(phi)))*dS

a_vel = dot(jump(v), un('+')*phi('+') - un('-')*phi('-') )*dS  + dot(v, un*phi)*ds

a = a_int + a_fac + a_vel

# Linear form
L = v*f*dx

# Set up boundary condition (apply strong BCs)
g = Expression("sin(pi*5.0*x[1])", degree=2)
bc = DirichletBC(V_dg, g, DirichletBoundary(), "geometric")

# Solution function
phi = Function(V_dg)

# Assemble and apply boundary conditions
A = assemble(a)
b = assemble(L)
bc.apply(A, b)

# Solve system
solve(A, phi.vector(), b)

# Project solution to a continuous function space
up = project(phi, V=V_cg)

file = File("temperature.pvd")
file << up

# Plot solution
plot(up)
plt.show()


In [ ]:

from dolfin import *
import matplotlib.pyplot as plt

# FIXME: Make mesh ghosted
parameters["ghost_mode"] = "shared_facet"

class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0] - 1.0) < DOLFIN_EPS and on_boundary

# Load mesh
mesh = UnitSquareMesh(50,50)

# Defining the function spaces
V_dg = FunctionSpace(mesh, "DG", 1)
V_cg = FunctionSpace(mesh, "CG", 1)
V_u  = VectorFunctionSpace(mesh, "CG", 2)

# Create velocity Function
u = Constant((3,3))

# Test and trial functions
v   = TestFunction(V_dg)
dphi = TrialFunction(V_dg)

# Solution function
phi = Function(V_dg)

# Diffusivity
kappa = Constant(0.0)

# Source term
f = Constant(0.0)

# Penalty term
alpha = Constant(5.0)

# Mesh-related functions
n = FacetNormal(mesh)
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2

# ( dot(v, n) + |dot(v, n)| )/2.0
un = (dot(u, n) + abs(dot(u, n)))/2.0

# Bilinear form
a_int = dot(grad(v), kappa*grad(phi) - u*phi)*dx

a_fac = kappa*(alpha/h('+'))*dot(jump(v, n), jump(phi, n))*dS \
      - kappa*dot(avg(grad(v)), jump(phi, n))*dS \
      - kappa*dot(jump(v, n), avg(grad(phi)))*dS

a_vel = dot(jump(v), un('+')*phi('+') - un('-')*phi('-') )*dS  + dot(v, un*phi)*ds

a = a_int + a_fac + a_vel

# Linear form
L = v*f*dx

# Set up boundary condition (apply strong BCs)
g = Expression("sin(pi*5.0*x[1])", degree=2)
bc = DirichletBC(V_dg, g, DirichletBoundary(), "geometric")

# Compute directional derivative about u in the direction of du (Jacobian)
dF = derivative (a - L, phi, dphi)

problem = NonlinearVariationalProblem (a - L, phi, bcs = bc, J = dF)

In [ ]:




# Assemble and apply boundary conditions
A = assemble(a)
b = assemble(L)
bc.apply(A, b)

# Solve system
solve(A, phi.vector(), b)

# Project solution to a continuous function space
up = project(phi, V=V_cg)

file = File("temperature.pvd")
file << up

# Plot solution
plot(up)
plt.show()


In [ ]:
""" Steady state advection-diffusion equation,
discontinuous formulation using full upwinding.

Implemented in python from cpp demo by Johan Hake.
"""

from dolfin import *
import matplotlib.pyplot as plt

# FIXME: Make mesh ghosted
parameters["ghost_mode"] = "shared_facet"

class DirichletBoundary_right (SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < (1.0 + DOLFIN_EPS) and on_boundary

class DirichletBoundary_left (SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < DOLFIN_EPS and on_boundary

# Load mesh
mesh = UnitSquareMesh (64, 64)

# Defining the function spaces
V_dg = FunctionSpace(mesh, "DG", 1)
V_cg = FunctionSpace(mesh, "CG", 1)
V_u  = VectorFunctionSpace(mesh, "CG", 1)

# Create velocity Function
u_const = Constant((1, 1))
u = interpolate(u_const, V_u)

# Test and trial functions
v   = TestFunction(V_dg)
phi = TrialFunction(V_dg)

# Diffusivity
kappa = Constant(0.0)

# Source term
f = Constant(0.0)

# Penalty term
alpha = Constant(5.0)

# Mesh-related functions
n = FacetNormal(mesh)
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2

# ( dot(v, n) + |dot(v, n)| )/2.0
un = (dot(u, n) + abs(dot(u, n)))/2.0

# Bilinear form
a_int = dot(grad(v), kappa*grad(phi) - u*phi)*dx

a_fac = kappa*(alpha/h('+'))*dot(jump(v, n), jump(phi, n))*dS \
      - kappa*dot(avg(grad(v)), jump(phi, n))*dS \
      - kappa*dot(jump(v, n), avg(grad(phi)))*dS

a_vel = dot(jump(v), un('+')*phi('+') - un('-')*phi('-') )*dS  + dot(v, un*phi)*ds

a = a_int + a_fac + a_vel

# Linear form
L = v*f*dx

# Set up boundary condition (apply strong BCs)
g_right = Expression("sin(pi*5.0*x[0])", degree=2)
g_left = Expression("sin(pi*5.0*x[0])", degree=2)

bc = DirichletBC(V_dg, g_right, DirichletBoundary_right(), "geometric")

# Solution function
phi_h = Function(V_dg)

# Assemble and apply boundary conditions
A = assemble(a)
b = assemble(L)
bc.apply(A, b)

# Solve system
solve(A, phi_h.vector(), b)

# Project solution to a continuous function space
up = project(phi_h, V=V_cg)

file = File("temperature.pvd")
file << up

# Plot solution
plot(up)
plt.show()

print("Solution vector norm (0): {!r}".format(phi_h.vector().norm("l2")))

In [ ]:
""" Steady state advection-diffusion equation,
discontinuous formulation using full upwinding.

Implemented in python from cpp demo by Johan Hake.
"""

from dolfin import *
import matplotlib.pyplot as plt

# FIXME: Make mesh ghosted
parameters ["ghost_mode"] = "shared_facet"

class DirichletBoundary_right (SubDomain):
    def inside (self, x, on_boundary):
        return x [0] < (1.0 + DOLFIN_EPS) and on_boundary

class DirichletBoundary_left (SubDomain):
    def inside (self, x, on_boundary):
        return x [0] < DOLFIN_EPS and on_boundary

# Load mesh
mesh = UnitSquareMesh (64, 64)

# Defining the function spaces
V_dg = FunctionSpace (mesh, "DG", 1)
V_cg = FunctionSpace (mesh, "CG", 1)
V_u  = VectorFunctionSpace (mesh, "CG", 1)

# Create velocity Function
u_const = Constant ((1, 1))
u = interpolate (u_const, V_u)

# Test and trial functions
v = TestFunction (V_dg)
dphi = TrialFunction (V_dg)
phi = Function (V_dg)

# Diffusivity
# kappa = Constant(0.0) #*activate for diffusion

# Source term
f = Constant (0.0)

# Penalty term
alpha = Constant (5.0)

# Mesh-related functions
n = FacetNormal (mesh)
h = CellDiameter (mesh)
h_avg = (h ('+') + h ('-'))/2

# Set up boundary condition (apply strong BCs)
g_right = Expression ("sin(pi*5.0*x [0])", degree = 2)
g_left = Expression ("sin(pi*5.0*x [0])", degree = 2)
bc = DirichletBC (V_dg, g_right, DirichletBoundary_right (), "geometric")

# ( dot(v, n) + |dot(v, n)| )/2.0
# un = (dot (u, n) + abs (dot (u, n)))/2.0 # Full upwinding

dot(grad(v),  u*phi)*dx

a = dot(grad(v), u*phi)*dx\
    - dot (avg (grad (v)), jump (phi, n))*dS \ 
    - dot (jump(v, n), avg (grad (phi)))*dS \
    + (alpha/h_avg)*dot (jump (v, n), jump (phi, n))*dS \
L = v*f*dx /
    - u0*dot (grad(v), n)*ds /
    + (gamma/h)*u0*v*ds

R = a - L

# Compute directional derivative about u in the direction of du (Jacobian)
dF = derivative (R, phi, dphi)
# Define transient solver function
problem = NonlinearVariationalProblem (R, phi, bcs = bc, J = dF)#, form_compiler_parameters = ffc_options)
transientsolver = NonlinearVariationalSolver (problem)
# Nonlinear solver
transientsolver.solve ()  

# Project solution to a continuous function space
up = project (phi, V = V_cg)

file = File ("temperature.pvd")
file << up

# Plot solution
plot (up)
plt.show ()

print ("Solution vector norm (0): {!r}".format (up.vector ().norm ("l2")))